In [1]:
import numpy as np
import pandas as pd
import os
import random
import re
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from sklearn import metrics
import nltk
from nltk.stem.porter import *
from nltk.tokenize import  word_tokenize
from textblob import Word
import gtts
from playsound import playsound
from deep_translator import GoogleTranslator
from nltk.sentiment.vader import SentimentIntensityAnalyzer
SIA = SentimentIntensityAnalyzer()

In [2]:
# Reading the data file and the database of the store
df = pd.read_csv('data.csv', index_col=0)
df = df.sample(frac=1)
db = pd.read_csv('database.csv', index_col=0)

In [3]:
# Spliting the dataset to train the model to predict the Intent from the sentences
X = df['Sentence']
y = df['Intent']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [4]:
# Function to predict the Intent from the sentences
def Predictions_Intent(X_test):
    text_clf_nb = Pipeline([('tfidf', TfidfVectorizer()), ('clf', MultinomialNB()),])
    text_clf_lsvc = Pipeline([('tfidf', TfidfVectorizer()), ('clf', LinearSVC()),])
    text_clf_nb.fit(X_train, y_train)
    x = text_clf_nb.predict(X_test)
    return x

In [5]:
# Showing the accuracy of the model
print(metrics.confusion_matrix(y_test, Predictions_Intent(X_test)))
print(metrics.classification_report(y_test, Predictions_Intent(X_test)))
print(metrics.accuracy_score(y_test, Predictions_Intent(X_test)))

[[  34    0   31]
 [   0  221    1]
 [   0    0 2997]]
                      precision    recall  f1-score   support

Asking for the price       1.00      0.52      0.69        65
  Check Availability       1.00      1.00      1.00       222
             unknown       0.99      1.00      0.99      2997

            accuracy                           0.99      3284
           macro avg       1.00      0.84      0.89      3284
        weighted avg       0.99      0.99      0.99      3284

0.9902557856272838


In [6]:
# Spliting the dataset to train the model to predict the Item from the sentences
A = df['Sentence']
B = df['Item']
A_train, A_test, B_train, B_test = train_test_split(A, B, test_size=0.2)

In [7]:
# Function to test the accuracy of predicting the Item from the sentence
def Test_Items(A_test):
    items = []
    for k,v in A_test.iteritems():
        list = word_tokenize(v)
        I = ''
        for word in list:
            word = ' {} '.format(word)
            if len(re.findall(' .antalon | p.ntalon | pa.talon | pan.alon | pant.lon | panta.on | pantal.n | pantalo. ', word)) > 0:
                I = 'pantalon'
                break
            elif len(re.findall(' .hirt | shi.t | shir. ', word)) > 0:
                I = 'shirt'
                break
            elif len(re.findall(' .-shirt | t.shirt | t-.hirt | t-s.irt | t-sh.rt | t-shi.t | t-shir. ', word)) > 0:
                I = 't-shirt'
                break
            elif len(re.findall(' .acket | j.cket | ja.ket | jac.et | jack.t | jacke. ', word)) > 0:
                I = 'jacket'
                break
            elif len(re.findall(' .res+ | d.es+ | dr.s+ | dre.+ | dre.s ', word)) > 0:
                I = 'dress'
                break
            elif len(re.findall(' .kirt | sk.rt | ski.t | skir. ', word)) > 0:
                I = 'skirt'
                break
            elif len(re.findall(' .hoes | s.oes | sh.es | sho.s | shoe.* ', word)) > 0:
                I = 'shoes'
                break
            elif len(re.findall(' .ul+over | p.l+over | pu.+over | pul.over | pul+.ver | pul+o.er | pul+ov.r | pul+ove.', word)) > 0:
                I = 'pullover'
                break
            elif len(re.findall(' .at | h.t | ha. ', word)) > 0:
                I = 'hat'
                break
            elif len(re.findall(' .eans | j.ans | je.ns | jea.s | jean. ', word)) > 0:
                I = 'jeans'
                break
            elif len(re.findall(' .hort | s.ort | sho.t | shor. ', word)) > 0:
                I = 'short'
                break
            elif len(re.findall(' .ody | b.dy | bo.y | bod. ', word)) > 0:
                I = 'body'
                break
            elif len(re.findall(' .hemise | c.emise | ch.mise | che.ise | chem.se | chemi.e | chemis. ', word)) > 0:
                I = 'chemise'
                break
            else:
                I = "unknown"
        items.append(I)
        Items = np.array(items)
    return Items

In [8]:
# Showing the accuracy of the model
print(metrics.confusion_matrix(B_test, Test_Items(A_test)))
print(metrics.classification_report(B_test, Test_Items(A_test)))
print(metrics.accuracy_score(B_test, Test_Items(A_test)))

[[  38    0    1    0    0    0    0    0    0    0    0    0    0    0]
 [   0   36    1    0    0    0    0    0    0    0    0    0    0    0]
 [   0    0   44    0    0    0    0    0    0    0    0    0    0    0]
 [   0    0    1   34    0    0    0    0    0    0    0    0    0    0]
 [   0    0    5    0   30    0    0    0    0    0    0    0    0    0]
 [   0    0    4    0    0   34    0    0    0    0    0    0    0    0]
 [   0    0    2    0    0    0   30    0    0    0    0    0    0    1]
 [   0    0    3    0    0    0    0   30    0    0    0    0    0    0]
 [   0    0    2    0    0    0    0    0   41    0    0    0    0    0]
 [   0    0    3    0    0    0    0    0    0   50    0    0    0    0]
 [   0    0    5    0    0    0    0    0    0    0   30    0    0    0]
 [   0    0    1    0    0    0    0    0    0    0    0   35    0    0]
 [   0    0    3    0    0    0    0    0    0    0    0    0   29    0]
 [   0    0    0    0    0    0    0    0    0    0

In [9]:
# Spliting the dataset to train the model to predict the Color from the sentences
C = df['Sentence']
D = df['Color']
C_train, C_test, D_train, D_test = train_test_split(C, D, test_size=0.2)

In [10]:
# Function to test the accuracy of predicting the Item's Color from the sentence
def Test_Color(C_test):
    colors = []
    for k,v in C_test.iteritems():
        list = word_tokenize(v)
        C = ''
        for word in list:
            word = ' {} '.format(word)
            if len(re.findall(' .ed | r.d | re. ', word)) > 0:
                C = 'red'
                break
            elif len(re.findall(' .lue | b.ue | bl.e | blu. | bleu ', word)) > 0:
                C = 'blue'
                break
            elif len(re.findall(' .re+n | g.e+n | gr.+n | gre+. ', word)) > 0:
                C = 'green'
                break
            elif len(re.findall(' .el*ow | y.l*ow | ye.*ow | yel*.w | yel*o. ', word)) > 0:
                C = 'yellow'
                break
            elif len(re.findall(' .lack | b.ack | bl.ck | bla.k | blac. ', word)) > 0:
                C = 'black'
                break
            elif len(re.findall(' .hite | w.ite | wh.te | whi.e | whit. ', word)) > 0:
                C = 'white'
                break
            elif len(re.findall(' .ray | g.ay | gr.y | gra. ', word)) > 0:
                C = 'gray'
                break
            elif len(re.findall(' .ink | p.nk | pi.k | pin. ', word)) > 0:
                C = 'pink'
                break
            elif len(re.findall(' .urple | p.rple | pu.ple | pur.le | purp.e | purpl. ', word)) > 0:
                C = 'purple'
                break
            elif len(re.findall(' .range | o.ange | or.nge | ora.ge | oran.e | orang. ', word)) > 0:
                C = 'orange'
                break
            elif len(re.findall(' .rown | b.own | br.wn | bro.n | brow.', word)) > 0:
                C = 'brown'
                break
            elif len(re.findall(' .yan | c.an | cy.n | cya. ', word)) > 0:
                C = 'cyan'
                break
            else:
                C = "unknown"
        colors.append(C)
        Colors = np.array(colors)
    return Colors

In [11]:
# Showing the accuracy of the model
print(metrics.confusion_matrix(D_test, Test_Color(C_test)))
print(metrics.classification_report(D_test, Test_Color(C_test)))
print(metrics.accuracy_score(D_test, Test_Color(C_test)))

[[  34    0    0    0    0    0    0    0    0    0    0    0    0]
 [   0   33    0    0    0    0    0    0    0    0    0    0    0]
 [   0    0   27    0    0    0    0    0    0    0    0    0    0]
 [   0    0    0   44    0    0    0    0    0    0    0    0    0]
 [   0    0    0    0   40    0    0    0    0    0    0    0    0]
 [   1    0    0    0    0   31    0    0    0    0    0    0    0]
 [   0    0    0    0    0    0   36    0    0    0    0    0    0]
 [   0    0    0    0    0    0    0   40    0    0    0    0    0]
 [   0    0    0    0    0    0    0    0   27    0    0    0    0]
 [   0    0    0    0    0    0    0    0    0   44    0    0    0]
 [   0    0    0    0    0    1    0    0    0    0 2846    0    0]
 [   0    0    0    0    0    0    0    1    0    0    0   40    0]
 [   0    0    0    0    0    0    0    0    0    0    0    0   39]]
              precision    recall  f1-score   support

       black       0.97      1.00      0.99        34
      

In [12]:
# Function to get the Item from the user's input
def Get_Item(txt):
    list = word_tokenize(txt)
    I = ''
    for word in list:
        word = ' {} '.format(word)
        if len(re.findall(' .antalon | p.ntalon | pa.talon | pan.alon | pant.lon | panta.on | pantal.n | pantalo. ', word)) > 0:
            I = 'pantalon'
            break
        elif len(re.findall(' .hirt | shi.t | shir. ', word)) > 0:
            I = 'shirt'
            break
        elif len(re.findall(' .-shirt | t.shirt | t-.hirt | t-s.irt | t-sh.rt | t-shi.t | t-shir. | tshirt ', word)) > 0:
            I = 't-shirt'
            break
        elif len(re.findall(' .acket | j.cket | ja.ket | jac.et | jack.t | jacke. ', word)) > 0:
            I = 'jacket'
            break
        elif len(re.findall(' .res+ | d.es+ | dr.s+ | dre.+ | dre.s ', word)) > 0:
            I = 'dress'
            break
        elif len(re.findall(' .kirt | sk.rt | ski.t | skir. ', word)) > 0:
            I = 'skirt'
            break
        elif len(re.findall(' .hoes | s.oes | sh.es | sho.s | shoe.* ', word)) > 0:
            I = 'shoes'
            break
        elif len(re.findall(' .ul+over | p.l+over | pu.+over | pul.over | pul+.ver | pul+o.er | pul+ov.r | pul+ove.', word)) > 0:
            I = 'pullover'
            break
        elif len(re.findall(' .at | h.t | ha. ', word)) > 0:
            I = 'hat'
            break
        elif len(re.findall(' .eans | j.ans | je.ns | jea.s | jean. ', word)) > 0:
            I = 'jeans'
            break
        elif len(re.findall(' .hort | s.ort | sho.t | shor. ', word)) > 0:
            I = 'short'
            break
        elif len(re.findall(' .ody | b.dy | bo.y | bod. ', word)) > 0:
            I = 'body'
            break
        elif len(re.findall(' .hemise | c.emise | ch.mise | che.ise | chem.se | chemi.e | chemis. ', word)) > 0:
            I = 'chemise'
            break
        else:
            I = "unknown"
    return I

In [13]:
# Function to get the Item's Color from the user's input
def Get_Color(txt):
    list = word_tokenize(txt)
    C = ''
    for word in list:
        word = ' {} '.format(word)
        if len(re.findall(' .ed | r.d | re. ', word)) > 0:
            C = 'red'
            break
        elif len(re.findall(' .lue | b.ue | bl.e | blu. | bleu ', word)) > 0:
            C = 'blue'
            break
        elif len(re.findall(' .re+n | g.e+n | gr.+n | gre+. ', word)) > 0:
            C = 'green'
            break
        elif len(re.findall(' .el*ow | y.l*ow | ye.*ow | yel*.w | yel*o. ', word)) > 0:
            C = 'yellow'
            break
        elif len(re.findall(' .lack | b.ack | bl.ck | bla.k | blac. ', word)) > 0:
            C = 'black'
            break
        elif len(re.findall(' .hite | w.ite | wh.te | whi.e | whit. ', word)) > 0:
            C = 'white'
            break
        elif len(re.findall(' .ray | g.ay | gr.y | gra. ', word)) > 0:
            C = 'gray'
            break
        elif len(re.findall(' .ink | p.nk | pi.k | pin. ', word)) > 0:
            C = 'pink'
            break
        elif len(re.findall(' .urple | p.rple | pu.ple | pur.le | purp.e | purpl. ', word)) > 0:
            C = 'purple'
            break
        elif len(re.findall(' .range | o.ange | or.nge | ora.ge | oran.e | orang. ', word)) > 0:
            C = 'orange'
            break
        elif len(re.findall(' .rown | b.own | br.wn | bro.n | brow.', word)) > 0:
            C = 'brown'
            break
        elif len(re.findall(' .yan | c.an | cy.n | cya. ', word)) > 0:
            C = 'cyan'
            break
        else:
            C = "unknown"
    return C

In [14]:
# Function to check the availability of the Item in the database
def Check_Availability(Item_Name):
    if db[db['Item'] == Item_Name]['Pieces'].iloc[0] != 0:
        answer = "Yes, This item is Available"
    else:
        answer = "Sorry, Unfortunately this item isn't available right now, we will do our best to make it available as soon as possible"
    return answer

In [15]:
# Function to get the Price of the selected Item from database
def Find_price(Item_Name):
    price = db[db['Item'] == Item_Name]['Price'].iloc[0]
    answer = 'This Piece costs {} Pounds'.format(price)
    return answer

In [16]:
# Function to fix user's typos
def AutoCorrect(txt):
    list = word_tokenize(txt)
    new = []
    for token in list:
        word = Word(token)
        x = word.spellcheck()[0][0]
        new.append(x)
    return ' '.join(new)

In [17]:
# Function to Stem / Lemmatize the sentence
def Stemmer(sentence):
    ps = PorterStemmer()
    words = sentence.split()
    sent = []
    for word in words:
        s = ps.stem(word)
        sent.append(s)
    final_result = ' '.join(sent)
    return final_result

In [18]:
# Function to save every input to a text file, To improve the model later
def Get_Input(sent):
    with open("Inputs.txt", "a") as file:
        file.write('{}\n'.format(sent))
        # Inputs.close()

In [19]:
# Function to pronounce English sentences
def Voice_EN(sent, file_name):
    tts = gtts.gTTS(sent)
    tts.save("{}.mp3".format(file_name))
    playsound("{}.mp3".format(file_name))

In [20]:
# Function to pronounce Arabic sentences
def Voice_AR(sent, file_name):
    tts = gtts.gTTS(sent, lang="ar")
    tts.save("{}.mp3".format(file_name))
    playsound("{}.mp3".format(file_name))

In [21]:
# Function to translate from Arabic to English
def AE(text):
  return GoogleTranslator(source='ar', target='en').translate(text)

In [22]:
# Function to translate from English to Arabic
def EA(text):
  return GoogleTranslator(source='en', target='ar').translate(text)

In [23]:
def English_Bot():
    Bot = True
    User_Intent = 'unknown'
    User_Item = 'unknown'
    User_Color = 'unknown'
    print('Welcome, English Bot is here to help you')
    Voice_EN('Welcome, English Bot is here to help you', 'en')
    Greetings = ['hi', 'hello', 'hey', 'heey', 'heeey']
    Answers = ['Hi there', 'Welcome', 'Nice to see you here']
    while (Bot == True):
        msg = input().lower()
        Get_Input(msg)
        B = True
        
        for word in word_tokenize(msg):
            if word in Greetings:
                a = random.choice(Answers)
                print('Bot: {}'.format(a))
                Voice_EN(a, 'a')
                os.remove("a.mp3")
                break
        if 'sell' in msg:
            print("Bot: We're currently selling many kinds of clothes and shoes")
            Voice_EN("We're currently selling many kinds of clothes and shoes", 's')
            os.remove("s.mp3")
        if 'thanks' in msg or 'thank you' in msg:
            print("Bot: You're Welcome")
            Voice_EN("You're Welcome", 'b')
            os.remove("b.mp3")
            B = False
        if 'bye' in msg or 'good bye' in msg or 'goodbye' in msg:
            print('Bot: Bye, Take Care')
            Voice_EN("Bye, Take Care", 'c')
            os.remove("c.mp3")
            B = False
            Bot = False    

        if B == True:
            if Predictions_Intent([Stemmer(msg)])[0] == 'unknown':
                if User_Intent == 'unknown':
                    while User_Intent == 'unknown':
                        print('Bot: How exactly can i help you ?')
                        Voice_EN('How exactly can i help you', 'd')
                        os.remove("d.mp3")
                        x = input().lower()
                        Get_Input(x)
                        if 'sell' in x:
                            print("Bot: We're currently selling clothes and shoes")
                            Voice_EN("We're currently selling clothes and shoes", 's')
                            os.remove("s.mp3")
                            break
                        User_Intent = Predictions_Intent([Stemmer(x)])[0]
                        User_Item = Get_Item(Stemmer(x))
                        User_Color = Get_Color(Stemmer(x))
            else:
                User_Intent = Predictions_Intent([Stemmer(msg)])[0]

            if Get_Item(Stemmer(msg)) == 'unknown':
                if User_Item == 'unknown':
                    while User_Item == 'unknown':
                        print('Bot: Which item are you looking for ?')
                        Voice_EN('Which item are you looking for', 'e')
                        os.remove("e.mp3")
                        y = input().lower()
                        Get_Input(y)
                        User_Item = Get_Item(Stemmer(y))
                        User_Color = Get_Color(Stemmer(y))
            else:
                User_Item = Get_Item(Stemmer(msg))

            if Get_Color(Stemmer(msg)) == 'unknown':
                if User_Color == 'unknown':
                    while User_Color == 'unknown':
                        print('Bot: Which color are you looking for ?')
                        Voice_EN('Which color are you looking for', 'f')
                        os.remove("f.mp3")
                        z = input().lower()
                        Get_Input(z)
                        User_Color = Get_Color(Stemmer(z))
            else:
                User_Color = Get_Color(Stemmer(msg))

            if User_Intent == 'Check Availability':
                Item = '{} {}'.format(User_Color, User_Item)
                g = Check_Availability(Item)
                print('Bot: {}'.format(g))
                Voice_EN(g, 'g')
                os.remove("g.mp3")
                User_Intent = 'unknown'

            elif User_Intent == 'Asking for the price':
                Item = '{} {}'.format(User_Color, User_Item)
                h = Find_price(Item)
                print('Bot: {}'.format(h))
                Voice_EN(h, 'h')
                os.remove("h.mp3")
                User_Intent = 'unknown'
                
            else:
                Item = '{} {}'.format(User_Color, User_Item)
                g = Check_Availability(Item)
                print('Bot: {}'.format(g))
                Voice_EN(g, 'g')
                os.remove("g.mp3")
                h = Find_price(Item)
                print('Bot: {}'.format(h))
                Voice_EN(h, 'h')
                os.remove("h.mp3")
                
        print(Item)
        print('**************************************************')

In [24]:
def Arabic_Bot():
    Bot = True
    User_Intent = 'unknown'
    User_Item = 'unknown'
    User_Color = 'unknown'
    Greetings = ['hi', 'hello', 'hey', 'heey', 'heeey']
    Answers = ['Hi there', 'Welcome', 'Nice to see you here']
    print(EA('Welcome, Arabic Bot is here to help you'))
    Voice_AR(EA('Welcome, Arabic Bot is here to help you'), 'ar')
    while (Bot == True):       
        msg = AE(input()).lower()
        Get_Input(msg)
        B = True
        for word in word_tokenize(msg):
            if word in Greetings:
                a = random.choice(Answers)
                print(EA('Bot: {}'.format(a)))
                Voice_AR(EA(a), 'a')
                os.remove("a.mp3")
                break
        if 'sell' in msg:
            print(EA("Bot: We're currently selling clothes and shoes"))
            Voice_AR(EA("We're currently selling clothes and shoes"), 's')
            os.remove("s.mp3")
        if msg == 'thanks' or msg == 'thank you':
            b = EA("You're Welcome")
            print("Bot: {}".format(b))
            Voice_AR(b, 'b')
            os.remove("b.mp3")
            B = False
        elif msg == 'bye' or msg == 'good bye' or msg == 'goodbye':
            c = EA('Bye, Take Care')
            print('Bot: {}'.format(c))
            Voice_AR(c, 'c')
            os.remove("c.mp3")
            B = False
            Bot = False
        
        if B == True:
            if Predictions_Intent([Stemmer(msg)])[0] == 'unknown':
                if User_Intent == 'unknown':
                    while User_Intent == 'unknown':
                        d = EA('How exactly can i help you ?')
                        print(d)
                        Voice_AR(d, 'd')
                        os.remove("d.mp3")
                        x = AE(input()).lower()
                        Get_Input(x)
                        if 'sell' in msg:
                            print(EA("Bot: We're currently selling clothes and shoes"))
                            Voice_AR(EA("We're currently selling clothes and shoes"), 's')
                            os.remove("s.mp3")
                            break
                        User_Intent = Predictions_Intent([Stemmer(x)])[0]
                        User_Item = Get_Item(Stemmer(x))
                        User_Color = Get_Color(Stemmer(x))
            else:
                User_Intent = Predictions_Intent([Stemmer(msg)])[0]

            if Get_Item(Stemmer(msg)) == 'unknown':
                if User_Item == 'unknown':
                    while User_Item == 'unknown':
                        e = EA('Which item are you looking for ?')
                        print(e)
                        Voice_AR(e, 'e')
                        os.remove("e.mp3")
                        y = AE(input()).lower()
                        Get_Input(y)
                        User_Item = Get_Item(Stemmer(y))
                        User_Color = Get_Color(Stemmer(y))
            else:
                User_Item = Get_Item(Stemmer(msg))

            if Get_Color(Stemmer(msg)) == 'unknown':
                if User_Color == 'unknown':
                    while User_Color == 'unknown':
                        f = EA('Which color are you looking for ?')
                        print(f)
                        Voice_AR(f, 'f')
                        os.remove("f.mp3")
                        z = AE(input()).lower()
                        Get_Input(z)
                        User_Color = Get_Color(Stemmer(z))
            else:
                User_Color = Get_Color(Stemmer(msg))

            if User_Intent == 'Check Availability':
                Item = '{} {}'.format(User_Color, User_Item)
                g = EA(Check_Availability(Item))
                print('Bot: {}'.format(g))
                Voice_AR(g, 'g')
                os.remove("g.mp3")
                User_Intent = 'unknown'

            elif User_Intent == 'Asking for the price':            
                Item = '{} {}'.format(User_Color, User_Item)
                h = EA(Find_price(Item))
                print('Bot: {}'.format(h))
                Voice_AR(h, 'h')
                os.remove("h.mp3")
                User_Intent = 'unknown'
        print(Item)
        print('**************************************************')

In [ ]:
Bot = True
l = False
print("Bot: Welcome to XYZ shop, I'm here to help you ... Please Choose the language, En or Ar ?")
Voice_EN("Welcome to XYZ shop, I'm here to help you ... Please Choose the language, English or Arabic ?", 'welcome')
os.remove("welcome.mp3")
print(EA("Welcome to XYZ shop, I'm here to help you ... Please Choose the language, English or Arabic ?"))
Voice_AR(EA("Welcome to XYZ shop, I'm here to help you ... Please Choose the language, English or Arabic ?"), 'Merhaba')
os.remove("Merhaba.mp3")

while l == False:
    ip = input().lower() 
    if ip == 'en' or ip == 'english':
        English_Bot()
        l = True
    elif ip == 'ar' or ip == 'arabic':
        Arabic_Bot()
        l = True
    else:
        print('Please choose an available language, type EN for English or AR for Arabic')
        Voice_EN('Please choose an available language, type EN for English or AR for Arabic', 'l')
        os.remove("l.mp3")
        l = False

Bot: Welcome to XYZ shop, I'm here to help you ... Please Choose the language, En or Ar ?
مرحبًا بكم في متجر XYZ ، أنا هنا لمساعدتك ... الرجاء اختيار اللغة ، الإنجليزية أو العربية؟
ar
مرحبًا ، برنامج الروبوت العربي هنا لمساعدتك
عندكوا فساتين ؟
ما هو اللون الذي تبحث عنه؟
سواد
Bot: نعم ، هذا العنصر متاح
black dress
**************************************************
عامل كم ؟
Bot: هذه القطعة سعرها 760 جنيه
black dress
**************************************************
